In [135]:
!pip install elasticsearch[async]==7.17.0 -q
!pip install elasticsearch-dsl==7.4.0 -q

In [594]:
import sys

sys.path.append('/app')

In [595]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd
import ast
import json

In [596]:
from memlish.config import ESTag

In [597]:
client = Elasticsearch("http://ec2-34-245-85-126.eu-west-1.compute.amazonaws.com:9200")
index = "fluentd-*"

In [598]:
# client.delete_by_query(index=[index], body={"query": {"match_all": {}}}, size=1000000)

In [599]:
# client.delete_by_query(index=[index], body={"query": {
#     "bool": {
#       "must": {
#         "match": {
#           "log": ESTag.TIMELOG
#         }
#       }
#     }
#   }}, size=1000000)

In [600]:
# time_logs = []
# time_log_search = Search(using=client, index=index).query("multi_match", query=ESTag.TIMELOG, fields=['log'])

# for hit in time_log_search.scan():
#     _hit = {}
#     _hit = ast.literal_eval(hit.to_dict()['log'])
#     time_logs.append(_hit)

In [601]:
client.info()

{'name': 'elastic',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'oyf18TppTqyj5cwfBqvswg',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [602]:
answ_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_ANSWER, fields=['log'])
choice_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_CHOICE, fields=['log'])

In [603]:
#cand_id - is guid. Example: 3940172d-fda5-4e02-b284-6f5e2074454c 
def find_choice_for_answer(cand_id, choices):
    for i in choices:
        choice_id = i['update']['result_id'].split('_')[1]
        if choice_id == cand_id:
            return i

In [604]:
choice_logs = []
rows = []

for hit in choice_search.scan():
    _hit = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit['update'] = json.loads(_hit['update'])
    _hit["timestamp"] = hit["@timestamp"]
    choice_logs.append(_hit)

for hit in answ_search.scan():
    result_row = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit["answer_timestamp"] = hit["@timestamp"]
    if len(_hit['template_options']) > 0:
        _hit_template_uniq_id = _hit['template_options'][0]['id'].split('_')[1]
        possible_choice = find_choice_for_answer(_hit_template_uniq_id, choice_logs)
    
    update = json.loads(_hit['update'])
    
    if possible_choice is not None:
        choice_update = possible_choice['update']
        result_row["template_choice"] = choice_update["result_id"]
        result_row["choice_timestamp"] = possible_choice['timestamp']
    
    result_row["answer_timestamp"] = _hit["answer_timestamp"]
    result_row["id"] = update["id"]
    result_row["user_id"] = update["from"]["id"]
    result_row["first_name"] = update["from"]["first_name"]
    result_row["last_name"] = update["from"].get("last_name", "")
    
    if update["from"].get("username", None) is not None:
        result_row["username"] = update["from"]["username"]
    else:
        result_row["username"] = update["from"]["id"]
        
    result_row["language_code"] = update["from"]["language_code"]
    result_row["chat_type"] = update["chat_type"]
    result_row["query"] = update["query"]
    result_row["candidates"] = _hit["template_options"]
    
    rows.append(result_row)

In [605]:
logs_df = pd.DataFrame(rows)

In [608]:
logs_df.head(2)

,answer_timestamp,id,user_id,first_name,last_name,username,language_code,chat_type,query,candidates,template_choice,choice_timestamp
0,2022-02-20T00:00:33.000000000+00:00,3159827879680430741,735704759,lukas,,lukasssss,en,group,cum,[{'id': '0_ca6ce419-dec9-4f7b-a698-e17b51b5d66...,NaN,NaN
1,2022-02-17T07:10:38.000000000+00:00,1409196998664532468,328104244,Volodymyr,Prypeshnyuk,ParadiseN,ru,supergroup,halfs don't coun,[{'id': '0_473b7a22-aaa4-4683-a42e-31dba887fe2...,NaN,NaN


In [609]:
banned_users = ['daniel_kovalenko', 'ParadiseN', 'vprypeshniuk', 'kilianovski', 'ostap_hembara', 'CheshireDog1865', 'dasha_hatsosh', 'NickKlimenko', 'mariupolec']

banned_mask = logs_df['username'].apply(lambda x: x not in banned_users)

with_choice_mask = logs_df['template_choice'].notna()

selection_mask = banned_mask & with_choice_mask

In [610]:
logs_df[selection_mask]['username'].value_counts()

mochi_chim_chim_chimmy        9
TheJosee                      7
Cycoric                       5
Anayilos_M                    5
ellegado                      5
sergej_havrylyuk              4
Three_part_baton              3
lukasssss                     3
702299132                     3
Dasun616                      3
Gifk0                         2
An7honym                      2
De_Percy_Jackson              2
Jared_Junior                  2
1241331520                    2
AmV1488                       2
kingchoudhry                  2
1472269408                    2
lexashott                     2
art_unltd                     2
922523433                     1
1270542385                    1
Arbazkhan6001                 1
Wooooooooooooooooooooooooo    1
Player_unknown_777            1
imameme_guy                   1
Super_trolly                  1
MarynaMarchenko               1
Oh_Ki                         1
ImruYas                       1
Anvilondre                    1
Yeabikib

In [611]:
post_date = pd.to_datetime('2022-02-18T15:30:00.000000000+00:00')

logs_sorted_dates = logs_df[selection_mask]['answer_timestamp']

logs_sorted_dates = pd.to_datetime(logs_sorted_dates)

after_post_mask = logs_sorted_dates > post_date

In [612]:
logs_df[selection_mask & after_post_mask]['username'].value_counts()

mochi_chim_chim_chimmy        8
TheJosee                      7
Cycoric                       5
Anayilos_M                    5
ellegado                      5
702299132                     3
Dasun616                      3
lukasssss                     3
De_Percy_Jackson              2
1472269408                    2
AmV1488                       2
1241331520                    2
Jared_Junior                  2
Gifk0                         2
An7honym                      2
kingchoudhry                  2
art_unltd                     2
lexashott                     2
ItsmehStk                     1
hedonist_01                   1
Arbazkhan6001                 1
Wooooooooooooooooooooooooo    1
prince_an                     1
Super_trolly                  1
supernaut911                  1
Oh_Ki                         1
ImruYas                       1
EmdoPL                        1
nihonnnn                      1
Anonchar                      1
922523433                     1
MT_6501 

In [613]:
logs_df[selection_mask & after_post_mask]['username'].shape

(80,)

In [614]:
logs_df[selection_mask & after_post_mask]['username'].unique().shape

(39,)